In [4]:
import pandas as pd
import os
import numpy as np
from dataclasses import dataclass
import random
import time



np.random.seed(14)

In [5]:
@dataclass
class Particle:
    x: float
    y: float
    theta: float
    weight: float 
        


@dataclass
class LandMark:
    x: float
    y: float
    index: int

In [6]:
gt_data = pd.read_csv('data/gt_data.txt', names=['X','Y','Orientation'], sep=' ')
map_data = pd.read_csv('data/map_data.txt', names=['X','Y','# landmark'])
control_data = pd.read_csv('data/control_data.txt', names=['velocity','Yaw rate'], sep=' ')

#observation = pd.read_csv('data/observation/observations_000001.txt', names = ['X cord','Y cord'], sep=' ')


result = [(x,y, landmark) for x,y,landmark in zip(map_data['X'],map_data['Y'], map_data['# landmark'])]
landarkList=[]
for res in result:
    l = LandMark(res[0],res[1],res[2])
    landarkList.append(l)

    

a = os.listdir("data/SensorDataFiles")
a.sort()
observation=[]
for i in range(len(a)):
    fileName = 'data/SensorDataFiles/'+a[i]
    observationTmp = pd.read_csv(fileName, names = ['X cord','Y cord'], sep=' ')
    observation.append(observationTmp)

In [47]:
def calculateDistance(landmark1, landmark2):
    a =  np.sqrt((landmark1.x - landmark2.x)**2 + (landmark1.y - landmark2.y)**2)
    return a

def findClosestLandmark(map_landmarks, singleObs):
    #todo: write code:
    # take the single obs and convert to global coordinates
    
    closest_landmark = map_landmarks[0]
    # check which landmark is the closest
    for i in range(len(map_landmarks)):
        if calculateDistance(singleObs, map_landmarks[i]) < calculateDistance(singleObs, closest_landmark):
            closest_landmark = map_landmarks[i]
    return closest_landmark
    
    
    
        
def getError(gt_data, bestParticle):
    error1 = np.abs(gt_data[0] - bestParticle.x)
    error2 = np.abs(gt_data[1] - bestParticle.y)
    error3 = np.abs(gt_data[2] - bestParticle.theta)
    if(error3>2*np.pi):
        error3 = 2*np.pi - error3
    return (error1, error2, error3)

def findObservationProbability(closest_landmark,map_coordinates, sigmaX, sigmaY):
    
    mew_x = closest_landmark.x
    mew_y = closest_landmark.y
    
    x = map_coordinates.x;
    y = map_coordinates.y;
    
    frac =  (1/ (2 * np.pi * sigmaX * sigmaY ))
    weight1 = (x-mew_x)**2/((sigmaX)**2)  +(y-mew_y)**2/(sigmaY**2)    
    ans = frac * np.exp(-1*weight1)
    return ans



def mapObservationToMapCoordinates(observation, particle):
    x = observation.x
    y = observation.y

    xt = particle.x
    yt = particle.y
    theta = particle.theta

    MapX = x * np.cos(theta) - y * np.sin(theta) + xt
    MapY = x * np.sin(theta) + y * np.cos(theta) + yt
    return MapX, MapY

def mapObservationsToMapCordinatesList(observations, particle):
    
    convertedObservations=[]
    i=0
    for obs in observations.iterrows():
        singleObs = LandMark(obs[1][0],obs[1][1],1)
        mapX, mapY = mapObservationToMapCoordinates(singleObs, particle)
        tmpLandmark = LandMark(x=mapX, y=mapY, index=i)
        i+=1
        convertedObservations.append(tmpLandmark)
    return convertedObservations



In [121]:
class ParticleFilter:
    particles = []
    def __init__(self, intialX, initialY, std, numOfParticles):
        self.number_of_particles = numOfParticles
        for i in range(self.number_of_particles):
           # tmpParticle = Particle(intialX,initialY,0,1)
            x = random.gauss(intialX, std)
            y = random.gauss(initialY, std)
            theta = random.uniform(0, 2*np.pi)
            tmpParticle = Particle(x,y , theta,1)
            self.particles.append(tmpParticle)



    def moveParticles(self, velocity, yaw_rate, delta_t=0.1):
        
        for i in range(self.number_of_particles):
            if(yaw_rate!=0):
                theta = self.particles[i].theta
                newTheta = theta + delta_t*yaw_rate;
                newX =  self.particles[i].x +(velocity/yaw_rate)*(np.sin(newTheta)-np.sin(theta));
                newY =  self.particles[i].y +(velocity/yaw_rate)*(np.cos(theta)-np.cos(newTheta));

                #todo Add noise!!
                self.particles[i].x = newX +random.gauss(0, 0.3)
                self.particles[i].y = newY +random.gauss(0, 0.3)
                self.particles[i].theta = newTheta +random.gauss(0, 0.01)
            else:
                print("ZERO!!!")
                
                
    
    def UpdateWeight(self, observations):
        
        for i in range(self.number_of_particles):
            map_obser = mapObservationsToMapCordinatesList(observations,self.particles[i])
            prob = 1.00000000000000000000000000000
            #print(self.particles[i])
            for obs in map_obser:
                    #print(obs)
                    land = findClosestLandmark(landarkList,obs)
                    #print(land)
                    prob *= findObservationProbability(land, self.particles[i], abs(land.x - obs.x),abs(land.y-obs.y))
            #print(prob)
            self.particles[i].weight = prob
        
    def getBestParticle(self):
        best_particle =  max(self.particles, key= lambda particle: particle.weight)
        return best_particle    

    def getBestParticleOut(self):
        x=0
        y=0
        theta=0
        for i in range(self.number_of_particles):
            x+= self.particles[i].x
            y+= self.particles[i].y
            theta+= self.particles[i].theta
        x=x/self.number_of_particles
        y=y/self.number_of_particles
        theta=theta/self.number_of_particles
        best_particle =  Particle(x,y,theta, weight=1)
        return best_particle        

    def PrintWeights(self):
        for i in range(self.number_of_particles):
            print("Weight:",self.particles[i].weight, self.particles[i].x,self.particles[i].y)

    def Resample(self):
        N = self.number_of_particles
        sumWeights = 0.0
        weights= []
        
        for i in range(N):
            sumWeights+=self.particles[i].weight
        
        for i in range(N):
            self.particles[i].weight = round(self.particles[i].weight / sumWeights,20)
            weights.append(self.particles[i].weight)
        particles_new = []
        
        for i in range(N):
            particles_new.append(np.random.choice(self.particles,1,weights))
        self.particles.clear()
        print(particles_new[0][0])
        for i in range(N):
            self.particles.append(particles_new[i][0])

In [122]:
sigmaY = 0.3

magicNumberOfParticles = 200


start = time.time()

def main():
    particleFilter = ParticleFilter(0 ,0 ,0.3, numOfParticles=magicNumberOfParticles)
    #articleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=magicNumberOfParticles)
   
    for i in range(len(observation)):
        #prediction
        if(i!=0):
            velocity = control_data.iloc[i-1][0]
            yaw_rate = control_data.iloc[i-1][1]
            particleFilter.moveParticles(velocity, yaw_rate)
        
        
        a = observation[i].copy()
        particleFilter.UpdateWeight(a)
        particleFilter.Resample()
        bestP = particleFilter.getBestParticle()
        print(bestP)
        error = getError(gt_data.iloc[i], bestP)
        print(i,error)
    end = time.time()
    print(end - start)
        
main()

Particle(x=0.056985208336686716, y=-0.6501352056646014, theta=2.508166923180823, weight=0.0)
Particle(x=0.1977337288808461, y=-0.11280814905079016, theta=0.9829560578667031, weight=0.9917666367179381)
0 (6.080766271119154, 2.07260814905079, 0.9829560578667031)
Particle(x=0.13144185582357454, y=-0.4860748048896956, theta=4.795608605292506, weight=0.0)
Particle(x=0.4567287265687462, y=0.27107271773894553, theta=1.2816004657475104, weight=0.870864891025777)
1 (6.206471273431253, 1.8114272822610544, 0.9722304657475105)
Particle(x=-0.3747448059329862, y=1.1853958562626103, theta=1.9309214533260948, weight=0.0)
Particle(x=0.6679715976263307, y=0.49840143790784774, theta=1.2839105422687755, weight=0.9999999999999996)
2 (6.3874284023736685, 1.7091985620921522, 0.9753505422687756)
Particle(x=3.9028037368740436, y=3.608194047280672, theta=0.8783620023539006, weight=0.0)
Particle(x=1.0748117881468802, y=0.8081774277326814, theta=1.2823250895224785, weight=1.0)
3 (6.38128821185312, 1.5276225722673

<ipython-input-121-74df6385d941>:78: RuntimeWarning: overflow encountered in double_scalars
  self.particles[i].weight = round(self.particles[i].weight / sumWeights,20)


Particle(x=-0.8340755236743524, y=-5.123770544658624, theta=4.93566407869344, weight=0.0)
Particle(x=0.8590275433266188, y=-6.893059546996705, theta=4.9034822978922925, weight=inf)
4 (7.006572456673381, 9.360159546996705, 4.593262297892292)


KeyboardInterrupt: 

In [110]:
(x=-0.43123262851514604, y=-0.042818418083184644, theta=5.568413235663596, weight=1

SyntaxError: invalid syntax (<ipython-input-110-13fecd66dbda>, line 1)

In [ ]:
import cv2

In [ ]:
# WIDTH=1600
# HEIGHT=1200

# def mouseCallback(event, x, y, flags,null):
#     global i
    

# WINDOW_NAME="Particle Filter"
# img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
# cv2.namedWindow(WINDOW_NAME)
# DELAY_MSEC=50
# #cv2.setMouseCallback(WINDOW_NAME,mouseCallback)

# i=1


    

# particleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=50)

# while(1):
 
#     cv2.imshow(WINDOW_NAME,img)
#     img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
    
#     if(i>2000):
#         break
    
#     for landmark in landarkList:
#         cv2.circle(img,tuple((int(landmark.x)*2+400, int(landmark.y)*2+300)),3,(255,0,0),-1)
    
#     #draw_particles:
#     for particle in particleFilter.particles:
#         cv2.circle(img,tuple((int(particle.x)*2+400,int(particle.y)*2+300)),1,(255,255,255),-1)
#     cv2.circle(img,tuple((int(gt_data.iloc[i].X)*2+400,int(gt_data.iloc[i].Y)*2+300)),1,(0,255,0),-1)
    
#     if cv2.waitKey(DELAY_MSEC) & 0xFF == 27:
#         break
#     if cv2.waitKey(33) == ord('a'):
#         velocity = control_data.iloc[i-1][0]
#         yaw_rate = control_data.iloc[i-1][1]
#         particleFilter.moveParticles(velocity, yaw_rate)
#         a = observation[i].copy()
#         particleFilter.UpdateWeight(a)
#         particleFilter.PrintWeights()
#         particleFilter.Resample()
        
#         i+=1

# cv2.destroyAllWindows()


In [68]:
from math import *
import random
import sys
print(sys.version)

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

class robot:
	def __init__(self):
		self.x = random.random()*world_size
		self.y = random.random()*world_size
		self.orientation = random.random()*2.0*pi
		self.forward_noise = 0.0
		self.turn_noise    = 0.0
		self.sense_noise   = 0.0


	def set(self,new_x,new_y,new_orientation):
		if new_x < 0 or new_x >= world_size:
			raise ValueError ('x coordinate out of bound')

		if new_y< 0 or new_y>=world_size:
			raise valueError ('Y coordinate out of bound')
		if new_orientation < 0 or new_orientation >= 2*pi:
			raise ValueError ('Orientation must be in [0..2pi]')

		self.x = float(new_x)
		self.y = float(new_y)
		self.orientation = float(new_orientation)

	def set_noise(self,new_f_noise, new_t_noise, new_s_noise):

		self.forward_noise = float(new_f_noise);
		self.turn_noise = float(new_t_noise);
		self.sense_noise = float(new_s_noise);


	def sense(self):
		Z = []
		for i in range(len(landmarks)):
			dist = sqrt((self.x - landmarks[i][0])**2 + (self.y- landmarks[i][1]) **2)
			dist += random.gauss(0.0,self.sense_noise)
			Z.append(dist)
		return Z

	def move(self,turn,forward):
		if forward < 0:
			raise ValueError ('Robot cant move backwards')

		orientation = self.orientation + float(turn) + random.gauss(0.0,self.turn_noise)
		orientation %= 2*pi

		dist = float(forward) + random.gauss(0.0, self.forward_noise)
		x = self.x + (cos(orientation)*dist)
		y = self.y + (sin(orientation)*dist)
		x %= world_size
		y %= world_size


		res = robot()
		res.set(x,y,orientation)
		res.set_noise(self.forward_noise,self.turn_noise, self.sense_noise)
		return res

	def Gaussian(self,mu,sigma,x):

		return exp(-((mu-x)**2)/(sigma**2)/2.0)/sqrt(2.0*pi*(sigma**2))

	def measurement_prob(self,measurement):

		prob = 1.0;
		for i in range(len(landmarks)):
			dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
			prob *= self.Gaussian(dist, self.sense_noise, measurement[i])

		return prob

	def __repr__(self):
		return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))	

def eval(r,p):
	sum = 0.0;
	for i in range(len(p)):
		dx  = (p[i].x-r.x+(world_size/2.0))% world_size - (world_size/2.0)
		dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
		err = sqrt(dx * dx + dy * dy)
		sum += err
	return sum / float(len(p))	


####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####

myrobot = robot()
# enter code here
forward_noise = 5.0
turn_noise = 0.1
sense_noise = 5.0
myrobot.set_noise(5.0,0.1,5.0)

myrobot.set(30.0, 50.0, pi/2)
myrobot = myrobot.move(-pi/2, 15.0)
print (myrobot.sense())
myrobot = myrobot.move(-pi/2, 10.0)
print (myrobot.sense())


# Now we want to create particles,
# p[i] = robot(). In this assignment, write
# code that will assign 1000 such particles
# to a list.
#
# Your program should print out the length
# of your list (don't cheat by making an
# arbitrary list of 1000 elements!)
#
# Don't modify the code below. Please enter
# your code at the bottom.

N = 1000
p = []

myrobot = robot()
myrobot = myrobot.move(0.1,5.0)
Z = myrobot.sense()

for i in range(N):
    x = robot()
    x.set_noise(0.05,0.05,5.0)
    p.append(x)


p2 = []
for i in range(N):
    p2.append(p[i].move(0.1,5.0))
p = p2

w = []

for i in range(N):
    w.append(p[i].measurement_prob(Z))
    
print(w)

3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
[42.53489438006441, 38.54916926868435, 29.335963341372597, 46.12496816812636]
[37.07039664993036, 53.72921967634844, 39.902844113175476, 43.62218354960201]
[1.4518329399436633e-65, 1.2664428853356738e-80, 7.162737315986077e-11, 3.4688705835540642e-65, 3.07708660805685e-50, 7.586605518428372e-104, 1.4208883200855177e-90, 1.6158034500682735e-35, 7.494769061248986e-70, 1.7409968119648017e-24, 4.395080253562828e-57, 1.0436964061507442e-36, 3.4100407180710682e-15, 6.132449466348067e-57, 2.9948366776429925e-05, 1.7157225455153022e-62, 6.711407973422057e-53, 2.5479826806874093e-40, 3.3668278848614133e-78, 2.4529782894774457e-65, 2.8689282779653165e-36, 2.335009608800375e-19, 7.0446266620901735e-84, 9.66871820479852e-52, 9.086800186720194e-50, 3.7322658158212395e-42, 4.238982625748885e-120, 4.0004892094435715e-54, 2.113280211353828e-29, 1.0924270274193823e-06, 6.616718137802358e-119, 6.270432221000622e-11, 2.3155906204205007e-2

In [78]:
x = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.039649455035843e-97, 5.064947208130591e-108, 0.0, 0.0, 0.0, 0.0, 1.2414055740966645e-34, 0.0, 1.6497523474692282e-40, 5.3989510499808124e-46, 3.928142575028093e-47, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2350614152905986e-36, 0.0, 0.0, 0.0, 3.100011397431673e-141, 1.6037530577654745e-30, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9061829504711847e-161, 1.1779805945719379e-122, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.364397581246902e-13, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8294547321909446e-126, 0.0, 0.0, 0.0, 0.0, 0.0, 7.643470440010682e-174, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999942526, 6.118348816629143e-23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.620405752400198e-28, 0.0, 0.0, 5.659244570965748e-84, 0.0, 0.0, 0.0, 2.334556165710494e-179, 5.123657939365491e-97, 0.0, 0.0, 0.0, 2.012496840303461e-121, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2233571249755617e-180, 0.0, 8.219364639178914e-196, 0.0, 0.0, 0.0, 0.0, 2.815378633743977e-76, 0.0, 0.0, 0.0, 0.0, 4.481644790251881e-109, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0643581772866818e-178, 0.0, 0.0, 0.0, 0.0, 1.130735411850651e-141, 0.0, 0.0, 0.0, 0.0, 0.0, 7.714262818958622e-116, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.3631902463795497e-93, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0841457700671726e-41, 0.0, 0.0, 0.0, 0.0, 4.910994828671085e-12, 5.982786091380361e-204, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4577727725167164e-113]

In [93]:
>>> aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']
p = []
print(x[82])
for i in range(len(x)):
    p.append(i)
print(p)
>>> np.random.choice(p, 150, p=x)

0.9999999999942526
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]


array([82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82,
       82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82])